# VGG

## VGG块

VGG块与经典卷积神经网络一样，由一系列卷积层、激活函数和汇聚层组成，每一个VGG块只有卷积层个数、输入通道数和输出通道数不一样，所以，一个VGG块可以定义为：

In [10]:
import torch
import torch.nn as nn


def vgg_block(num_convs, in_channels, out_channels):
    layers = []
    for _ in range(num_convs):
        layers.append(nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1))
        layers.append(nn.ReLU())
        in_channels = out_channels
    layers.append(nn.MaxPool2d(kernel_size=2, stride=2))
    return nn.Sequential(*layers)

VGG神经网络就是由多个不同卷积层个数、输入通道数和输出通道数的VGG块共同组成卷积层，最后再连接3个全连接层组成整个神经网络，其[架构](../image/vgg.png)如图所示：
<div align="center">
<img src = "https://s2.loli.net/2022/04/12/lr2zdoiLY5qActh.png" width = "35%">
<br>AlexNet架构图（左）VGG块（中） VGG架构图（右）
</div>

基于pytorch实现AlexNet网络的模型搭建如下：

## 模型建立


In [11]:
def vggnet(conv, num_classes):
    conv_layers = []
    in_channels = 1
    # 调用VGG块定义卷积层模块
    for (num_convs, out_channels) in conv:
        conv_layers.append(vgg_block(num_convs, in_channels, out_channels))
        in_channels = out_channels

    net_vgg = nn.Sequential(
        *conv_layers,
        nn.Flatten(),
        nn.Linear(out_channels * 7 * 7, 4096),
        nn.ReLU(),
        nn.Dropout(p=0.5),
        nn.Linear(4096, 4096),
        nn.ReLU(),
        nn.Dropout(p=0.5),
        nn.Linear(4096, num_classes),
        )
    return net_vgg

In [12]:
conv11 = ((1, 64), (1, 128), (2, 256), (2, 512), (2, 512)) # VGG_11
conv13 = ((2, 64), (2, 128), (2, 256), (2, 512), (2, 512)) # VGG_13
conv16 = ((2, 64), (2, 128), (3, 256), (3, 512), (3, 512)) # VGG_16
conv19 = ((2, 64), (2, 128), (4, 256), (4, 512), (4, 512)) # VGG_19
num_classes = 1000

net = vggnet(conv16, num_classes)

## 模型检验

构建一个高度和宽度为224的单通道数据样本，观察每个层输出的形状

In [13]:
X = torch.randn(size=(1, 1, 224, 224))
for layer in net:
    X = layer(X)
    print(layer.__class__.__name__,'output shape: \t',X.shape)

Sequential output shape: 	 torch.Size([1, 64, 112, 112])
Sequential output shape: 	 torch.Size([1, 128, 56, 56])
Sequential output shape: 	 torch.Size([1, 256, 28, 28])
Sequential output shape: 	 torch.Size([1, 512, 14, 14])
Sequential output shape: 	 torch.Size([1, 512, 7, 7])
Flatten output shape: 	 torch.Size([1, 25088])
Linear output shape: 	 torch.Size([1, 4096])
ReLU output shape: 	 torch.Size([1, 4096])
Dropout output shape: 	 torch.Size([1, 4096])
Linear output shape: 	 torch.Size([1, 4096])
ReLU output shape: 	 torch.Size([1, 4096])
Dropout output shape: 	 torch.Size([1, 4096])
Linear output shape: 	 torch.Size([1, 1000])


在经过每一个VGG块后，图像高宽减半，通道数增加。从上面四个VGG模型可以看到，无论是VGG多少，都有5个VGG模块，而且这五个模块仅仅是num_convs不一样，每一个模块对应的out_channels是一样的。

由于VGG所含有的卷积层更多，网络更深，所以计算量比AlexNet更大，计算也慢很多。

## 模型训练
*后补*

## 模型测试
*后补*